In [ ]:
import pandas as pd
from IPython.display import display

red_wine = pd.read_csv('../input/red-wine-dataset/wineQualityReds.csv')
red_wine = red_wine.drop(red_wine.columns[[0]],axis=1)
red_wine.head()

In [ ]:
red_wine.info()

## creating training and validation splits

In [ ]:
df_train = red_wine.sample(frac=0.7, random_state=0)
df_valid = red_wine.drop(df_train.index)
display(df_train.head(4))

## scale to [0,1]

In [ ]:
max_ = df_train.max(axis=0)
min_ = df_train.min(axis=0)
df_train = (df_train - min_)/(max_ - min_)
df_valid = (df_valid - min_)/(max_ - min_)

## split features and target

In [ ]:
X_train = df_train.drop('quality',axis=1)
X_valid = df_valid.drop('quality',axis=1)
y_train = df_train['quality']
y_valid = df_valid['quality']
print(X_train.shape)
print(X_valid.shape)
print(y_train.shape)
print(y_valid.shape)

## Neural Network

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    #hidden ReLU units
    layers.Dense(units=512, activation='relu', input_shape=[11]),
    layers.BatchNormalization(),
    
    layers.Dropout(rate=0.3),#Drop out layer to prevent overfitting
    layers.Dense(units=512, activation='relu'),    
    layers.BatchNormalization(),
 
    layers.Dropout(rate=0.3),
    layers.Dense(units=512, activation='relu'),
    layers.BatchNormalization(),
 
    #linear output layer
    layers.Dense(units=1)
])

model.compile(
    optimizer = 'adam',
    loss = 'mae'
)

## Adding Early Stopping : we can stop training whenever it seems validation loss isnt decreasing

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    min_delta = 0.001, #min change to count improve
    patience = 20, #least epochs to wait
    restore_best_weights = True
)

## Fit Model

In [ ]:
history = model.fit(
    X_train,y_train,
    validation_data = (X_valid,y_valid),
    batch_size = 256,
    epochs = 500,
    callbacks = [early_stopping],
    verbose = 0
)

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.loc[:,['loss','val_loss']].plot()

print("Min validation loss:{}".format(history_df['val_loss'].min()))